# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

In [28]:
import pandas as pd
import numpy as np

recipes = pd.read_csv("recipes.tsv", sep="\t")
tags = pd.read_csv("recipe-tags.tsv", sep="\t")

r = pd.read_csv("ratings_template.tsv", sep=r"\s+|\t", engine="python", names=["recipe_slug","rating"], header=0)
r["rating"] = pd.to_numeric(r["rating"])
r["rating"] = (r["rating"] - 1) * 0.25  

r.to_csv("ratings.tsv", sep="\t", index=False)


## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [29]:
# YOUR CHANGES HERE

ratings = pd.read_csv("ratings.tsv", sep="\t")

tags_encode = (
    tags.dropna(subset=["recipe_slug","recipe_tag"])
        .drop_duplicates()
        .assign(value=1)
        .pivot_table(index="recipe_slug",
                     columns="recipe_tag",
                     values="value",
                     aggfunc="max",
                     fill_value=0))

tags_1hot = tags_encode.astype("int64")

order = ratings["recipe_slug"]
tags_1hot = tags_1hot.reindex(order).fillna(0).astype("int64")

features = pd.concat(
    [order.rename("recipe_slug"),
     pd.Series(1, index=order.index, name="bias"),
     tags_1hot.reset_index(drop=True)],
    axis=1)

features.to_csv("features.tsv", sep="\t", index=False)

Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [30]:
# YOUR CHANGES HERE
from sklearn.linear_model import Ridge

X = features.drop(columns=["recipe_slug"])
y = ratings["rating"]

model = Ridge(alpha=1, fit_intercept=False)
model.fit(X, y)

,alpha,1
,fit_intercept,False
,copy_X,True
,max_iter,None
,tol,0.0001
,solver,'auto'
,positive,False
,random_state,None


Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not add anything for the `intercept_` attribute of a scikit-learn model; this will be covered by the coefficient for the bias column added in part 2.

In [31]:
# YOUR CHANGES HERE

coeffs = pd.DataFrame({
    "recipe_tag": X.columns,
    "coefficient": model.coef_})

# Save to model.tsv
coeffs.to_csv("model.tsv", sep="\t", index=False)

Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [32]:
# YOUR CHANGES HERE

model_df = pd.read_csv("model.tsv", sep="\t")

X = features.drop(columns=["recipe_slug"])

model = (
    model_df.set_index("recipe_tag")["coefficient"]
            .reindex(X.columns)
            .fillna(0.0)
            .to_numpy())

score_estimate = X.to_numpy(dtype=float) @ model.astype(float)

estimates = pd.DataFrame({
    "recipe_slug": features["recipe_slug"],
    "score_estimate": score_estimate})

estimates.to_csv("estimates.tsv", sep="\t", index=False)

Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [33]:
# YOUR CHANGES HERE

alpha = 2.0
lambda_ = 1.0

X = features.drop(columns=["recipe_slug"]).to_numpy(dtype=float)

A = X.T @ X + lambda_ * np.eye(X.shape[1])
A_inv = np.linalg.inv(A)
beta = A_inv @ X.T @ y

sb = []
for i in range(X.shape[0]):
    x = X[i, :].reshape(-1, 1)
    pred = float(x.T @ beta)
    conf = alpha * np.sqrt(float(x.T @ A_inv @ x))
    sb.append(pred + conf)

UCB_bounds = pd.DataFrame({
    "recipe_slug": features["recipe_slug"],
    "score_bound": sb
})
UCB_bounds.to_csv("bounds.tsv", sep="\t", index=False)

/tmp/ipykernel_78599/368911800.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred = float(x.T @ beta)
/tmp/ipykernel_78599/368911800.py:16: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  conf = alpha * np.sqrt(float(x.T @ A_inv @ x))


Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

In [34]:
# YOUR CHANGES HERE

X_df = features.drop(columns=["recipe_slug"])
X = X_df.to_numpy(dtype=float)
slugs = features["recipe_slug"].tolist()
reward_map = dict(zip(ratings["recipe_slug"], ratings["rating"]))

T = min(100, len(slugs))

d = X.shape[1]
A_inv = np.linalg.inv(lambda_ * np.eye(d))
b = np.zeros(d)
theta = A_inv @ b

picked = set()
rows = []

for t in range(T):
    cand_idx = [i for i in range(len(slugs)) if i not in picked]
    XAinv = X[cand_idx] @ A_inv
    means = (XAinv @ b)
    confs = np.sqrt(np.sum(XAinv * X[cand_idx], axis=1))
    p = means + alpha * confs

    j_local = int(np.argmax(p))
    i = cand_idx[j_local]
    x = X[i]
    slug = slugs[i]
    reward = float(reward_map[slug])
    score_bound = float(p[j_local])

    rows.append({"recipe_slug": slug, "score_bound": score_bound, "reward": reward})
    picked.add(i)

    x_col = x.reshape(-1, 1)
    denom = 1.0 + (x_col.T @ A_inv @ x_col).item()
    A_inv = A_inv - (A_inv @ x_col @ x_col.T @ A_inv) / denom
    b = b + reward * x
    theta = A_inv @ b

recs = pd.DataFrame(rows, columns=["recipe_slug", "score_bound", "reward"])
recs.to_csv("recommendations.tsv", sep="\t", index=False)

Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.